# **Package Installations**

In [12]:
!pip3 install cdt
!pip3 install torch
!pip3 install networkx
!pip3 install causal-learn
!pip3 install statsmodels
!pip3 install xges
!pip3 install numba


[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.7 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 11.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 11.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4

[notice] A new release of 

# **Helper Functions**

In [9]:
#!/usr/bin/env python
"""
Complete Script to Compare PC and FLAR
across metrics (SID, SHD, ATE_RMSE, runtime) for variable counts 10, 50, and 100.
Evaluations are done for:
  • DAG types: Erdős–Rényi (ER) and Scale‑Free (SF)
  • SEM types: linear and non‑linear
  • Noise types: gaussian, exponential, and laplace

Make sure to install dependencies:
    pip install numpy pandas torch networkx causal-learn statsmodels
"""

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import networkx as nx
import time
import statsmodels.api as sm

# ------------------------------------------------
# 1. DAG Generation and Data Simulation Functions
# ------------------------------------------------

def generate_erdos_renyi_dag(num_nodes, edge_prob):
    """
    Generate an Erdős–Rényi random DAG with given edge probability.
    """
    perm = np.random.permutation(num_nodes)
    adjacency_matrix = np.zeros((num_nodes, num_nodes), dtype=int)
    for i in range(num_nodes):
        for j in range(i+1, num_nodes):
            if np.random.rand() < edge_prob:
                adjacency_matrix[perm[i], perm[j]] = 1
    return adjacency_matrix

def remove_all_cycles(G):
    """
    Remove edges from cycles until the graph is a DAG.
    Works in-place on a networkx DiGraph.
    """
    while True:
        try:
            cycle_edges = nx.find_cycle(G, orientation='original')
            for (u, v, _) in cycle_edges:
                G.remove_edge(u, v)
        except nx.NetworkXNoCycle:
            break
    return G

def generate_scale_free_dag(num_nodes):
    """
    Generate a scale-free DAG: first generate a scale-free network and then remove cycles.
    """
    G = nx.scale_free_graph(num_nodes, seed=None)
    G_simple = nx.DiGraph(G)  # convert to simple directed graph
    G_simple.remove_edges_from(nx.selfloop_edges(G_simple))
    G_dag = remove_all_cycles(G_simple.copy())
    adj_matrix = nx.to_numpy_array(G_dag, dtype=int)
    np.fill_diagonal(adj_matrix, 0)
    return adj_matrix

def simulate_sem_with_weights(adjacency_matrix, n_samples, noise_type='gaussian',
                              weight_scale=1.0, random_state=None):
    """
    Simulate data from a linear SEM:
         X_j = sum_{i in Pa(j)} W_{i,j} * X_i + noise_j
    Returns data X and the true weight matrix W.
    """
    if random_state is not None:
        np.random.seed(random_state)

    num_nodes = adjacency_matrix.shape[0]
    G = nx.DiGraph(adjacency_matrix)
    topo_order = list(nx.topological_sort(G))

    # Generate weights on edges
    W = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        for j in range(num_nodes):
            if adjacency_matrix[i, j] == 1:
                W[i, j] = np.random.normal(loc=0.0, scale=weight_scale)

    X = np.zeros((n_samples, num_nodes))
    for s in range(n_samples):
        for node in topo_order:
            parents = np.where(adjacency_matrix[:, node] == 1)[0]
            parents_sum = np.sum(W[parents, node] * X[s, parents])
            if noise_type.lower() == 'gaussian':
                noise = np.random.normal(loc=0.0, scale=1.0)
            elif noise_type.lower() == 'laplace':
                noise = np.random.laplace(loc=0.0, scale=1.0)
            elif noise_type.lower() == 'exponential':
                noise = np.random.exponential(scale=1.0)
            else:
                raise ValueError("noise_type must be 'gaussian', 'laplace', or 'exponential'.")
            X[s, node] = parents_sum + noise
    return X, W

def simulate_non_linear_sem_with_weights(adjacency_matrix, n_samples, noise_type='gaussian',
                                           weight_scale=1.0, random_state=None, non_linear_fn=np.tanh):
    """
    Simulate data from a non-linear SEM:
         X_j = f( sum_{i in Pa(j)} W_{i,j} * X_i ) + noise_j
    Returns data X and the true weight matrix W.
    """
    if random_state is not None:
        np.random.seed(random_state)

    num_nodes = adjacency_matrix.shape[0]
    G = nx.DiGraph(adjacency_matrix)
    topo_order = list(nx.topological_sort(G))

    # Generate weights on edges
    W = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        for j in range(num_nodes):
            if adjacency_matrix[i, j] == 1:
                W[i, j] = np.random.normal(loc=0.0, scale=weight_scale)

    X = np.zeros((n_samples, num_nodes))
    for s in range(n_samples):
        for node in topo_order:
            parents = np.where(adjacency_matrix[:, node] == 1)[0]
            parents_sum = np.sum(W[parents, node] * X[s, parents])
            non_linear_term = non_linear_fn(parents_sum)
            if noise_type.lower() == 'gaussian':
                noise = np.random.normal(loc=0.0, scale=1.0)
            elif noise_type.lower() == 'laplace':
                noise = np.random.laplace(loc=0.0, scale=1.0)
            elif noise_type.lower() == 'exponential':
                noise = np.random.exponential(scale=1.0)
            else:
                raise ValueError("noise_type must be 'gaussian', 'laplace', or 'exponential'.")
            X[s, node] = non_linear_term + noise
    return X, W

# ------------------------------------------------
# 2. Metrics: SHD, SID, and ATE_RMSE
# ------------------------------------------------

def shd(true_adj: np.ndarray, est_adj: np.ndarray) -> int:
    """Compute Structural Hamming Distance."""
    return int(np.sum(true_adj != est_adj))

def _compute_ancestors(adj: np.ndarray):
    """Helper for SID calculation."""
    G = nx.DiGraph(adj)
    d = adj.shape[0]
    ancestors_list = []
    for node in range(d):
        ancestors_list.append(set(nx.ancestors(G, node)))
    return ancestors_list

def sid(true_adj: np.ndarray, est_adj: np.ndarray) -> int:
    """Compute Structural Intervention Distance (SID)."""
    true_anc = _compute_ancestors(true_adj)
    est_anc = _compute_ancestors(est_adj)
    d = true_adj.shape[0]
    score = 0
    for j in range(d):
        diff_1 = true_anc[j].difference(est_anc[j])
        diff_2 = est_anc[j].difference(true_anc[j])
        score += len(diff_1) + len(diff_2)
    return score

def compute_total_effect_matrix(W: np.ndarray) -> np.ndarray:
    """Compute total effect matrix T = (I - W)^{-1} - I."""
    d = W.shape[0]
    I = np.eye(d)
    try:
        inv = np.linalg.inv(I - W)
    except np.linalg.LinAlgError:
        return np.zeros((d, d))
    return inv - I

def rmse_ate(W_true: np.ndarray, W_est: np.ndarray) -> float:
    """Compute RMSE of total causal effects (ATE RMSE)."""
    T_true = compute_total_effect_matrix(W_true)
    T_est  = compute_total_effect_matrix(W_est)
    return np.sqrt(np.mean((T_true - T_est) ** 2))

# **FLAR**

In [10]:
# ------------------------------------------------
# 3. FLAR Implementation
# ------------------------------------------------

def squared_loss(x_true, x_pred):
    return 0.5 * torch.mean((x_true - x_pred)**2)

def dag_constraint(W):
    d = W.shape[0]
    WW = W * W
    expm_WW = torch.matrix_exp(WW)
    h = torch.trace(expm_WW) - d
    return h

def apply_mask(W, mask):
    with torch.no_grad():
        W *= mask
        d = W.shape[0]
        for i in range(d):
            W[i, i] = 0.0

def binarize_adjacency(W, threshold=0.3):
    W_np = W.detach().cpu().numpy()
    W_bin = (np.abs(W_np) > threshold).astype(float)
    np.fill_diagonal(W_bin, 0.0)
    return W_bin

class WeakLearnerNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=40):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
    def forward(self, x):
        return self.net(x)

class FunctionalBoostingModel(nn.Module):
    def __init__(self, d, max_num_weak_learners=50, hidden_dim=40):
        super().__init__()
        self.d = d
        self.max_num_weak_learners = max_num_weak_learners
        self.hidden_dim = hidden_dim
        self.learners_for_var = [[] for _ in range(d)]
        self.weak_learners = nn.ModuleList()
        self.current_counts = [0]*d

    def forward(self, X, W):
        N, d = X.shape
        device = X.device
        Xhat = []
        for i in range(d):
            mask_row = W[i, :]
            masked_input = X * mask_row
            pred_i = torch.zeros((N, 1), dtype=X.dtype, device=device)
            for learner in self.learners_for_var[i]:
                pred_i += learner(masked_input)
            Xhat.append(pred_i)
        return torch.cat(Xhat, dim=1)

    def add_weak_learner(self, i):
        wl = WeakLearnerNN(input_dim=self.d, hidden_dim=self.hidden_dim)
        self.weak_learners.append(wl)
        self.learners_for_var[i].append(wl)
        self.current_counts[i] += 1

    def fit_new_weak_learner(self, i, X, residual_i, W, n_epochs=5, lr=0.01, verbose=False):
        self.add_weak_learner(i)
        wl = self.learners_for_var[i][-1]
        optimizer = optim.Adam(wl.parameters(), lr=lr)
        for epoch in range(n_epochs):
            optimizer.zero_grad()
            mask_row = W[i, :]
            masked_input = X * mask_row
            pred = wl(masked_input)
            loss = torch.mean((pred - residual_i)**2)
            loss.backward()
            optimizer.step()
            if verbose:
                print(f"    [WL-Fit Var {i} Ep {epoch}] Loss={loss.item():.6f}")

class DAGBoostingTrainer:
    def __init__(
        self,
        d,
        adjacency_mask=None,
        lr_W=0.01,
        lambda_h=5.0,
        alpha_init=0.0,
        max_iter=5,
        max_num_weak_learners=50,
        hidden_dim=40,
        tol=1e-4,
        patience=2,
        device=torch.device("cpu")
    ):
        self.d = d
        self.model = FunctionalBoostingModel(
            d=d,
            max_num_weak_learners=max_num_weak_learners,
            hidden_dim=hidden_dim
        ).to(device)
        # Initialize real-valued adjacency matrix W
        W_init = 0.01 * torch.randn(d, d, device=device)
        for i in range(d):
            W_init[i, i] = 0.0
        self.W = nn.Parameter(W_init)

        if adjacency_mask is None:
            adjacency_mask = np.ones((d, d), dtype=np.float32)
            np.fill_diagonal(adjacency_mask, 0.)
        self.adjacency_mask = torch.tensor(adjacency_mask, dtype=torch.float32, device=device)

        self.lambda_h = lambda_h
        self.alpha = alpha_init
        self.lr_W = lr_W
        self.max_iter = max_iter
        self.tol = tol
        self.patience = patience
        self.device = device
        self.best_loss = float('inf')
        self.no_improv_steps = 0
        self.stop_early = False

    def parameters(self):
        return list(self.model.parameters()) + [self.W]

    def apply_domain_mask_and_no_loops(self):
        apply_mask(self.W, self.adjacency_mask)

    def augmented_lagrangian_loss(self, X):
        Xhat = self.model(X, self.W)
        recon = squared_loss(X, Xhat)
        h_val = dag_constraint(self.W)
        aug = self.alpha * h_val + 0.5 * self.lambda_h * (h_val ** 2)
        return recon + aug, recon, h_val

    def update_dual(self, h_val):
        self.alpha = self.alpha + self.lambda_h * h_val.item()

    def train(self, X, batch_size=512, n_inner_epochs=10, fit_new_learner_epochs=5, verbose=True):
        if not isinstance(X, torch.Tensor):
            X = torch.tensor(X, dtype=torch.float32, device=self.device)
        else:
            X = X.to(self.device)

        N_full = X.shape[0]

        for outer_iter in range(self.max_iter):
            if verbose:
                print(f"\n===== Outer Iteration {outer_iter+1}/{self.max_iter} =====")
            indices = np.random.permutation(N_full)
            subset_idx = indices[:batch_size]
            X_sub = X[subset_idx]

            with torch.no_grad():
                Xhat_sub = self.model(X_sub, self.W)
                residuals_sub = X_sub - Xhat_sub
                mse_val = torch.mean((residuals_sub)**2).item()
            if verbose:
                print(f"  Sub-batch size={batch_size}, MSE before new learners: {mse_val:.6f}")

            d = X_sub.shape[1]
            for i in range(d):
                if self.model.current_counts[i] < self.model.max_num_weak_learners:
                    residual_i_sub = residuals_sub[:, i:i+1]
                    self.model.fit_new_weak_learner(i=i, X=X_sub, residual_i=residual_i_sub,
                                                     W=self.W, n_epochs=fit_new_learner_epochs,
                                                     lr=0.01, verbose=False)

            # Update W via inner epochs
            opt = optim.Adam(self.parameters(), lr=self.lr_W)
            for epoch in range(n_inner_epochs):
                opt.zero_grad()
                loss_total, loss_recon, h_val = self.augmented_lagrangian_loss(X_sub)
                loss_total.backward()
                opt.step()
                self.apply_domain_mask_and_no_loops()

            with torch.no_grad():
                _, _, h_val = self.augmented_lagrangian_loss(X_sub)
            self.update_dual(h_val)

            with torch.no_grad():
                Xhat_sub = self.model(X_sub, self.W)
                mse_val = torch.mean((X_sub - Xhat_sub)**2).item()
                h_now = dag_constraint(self.W).item()
            if verbose:
                print(f"  [Iteration {outer_iter+1}] MSE={mse_val:.6f}, h(W)={h_now:.6f}, alpha={self.alpha:.3f}")
            if mse_val < self.best_loss - self.tol:
                self.best_loss = mse_val
                self.no_improv_steps = 0
            else:
                self.no_improv_steps += 1
            if self.no_improv_steps >= self.patience:
                if verbose:
                    print("No improvement; early stopping.")
                self.stop_early = True
                break
        return self.W.detach(), self.model

    def get_binarized_adjacency(self, threshold=0.3):
        return binarize_adjacency(self.W, threshold=threshold)

def run_dagboost_method(X_data, threshold=0.3, max_iter=5, max_num_weak_learners=50,
                        hidden_dim=40, lambda_h=5.0, verbose=False):
    """
    Run the DAG‑Boosting (FLAR) method and return:
       - final binary estimated adjacency matrix, and
       - the trainer (for further access to model/W)
    """
    device = torch.device("cpu")
    d = X_data.shape[1]
    adjacency_mask = np.ones((d, d), dtype=np.float32)
    np.fill_diagonal(adjacency_mask, 0.0)
    trainer = DAGBoostingTrainer(
        d=d,
        adjacency_mask=adjacency_mask,
        lr_W=0.01,
        lambda_h=lambda_h,
        alpha_init=0.0,
        max_iter=max_iter,
        max_num_weak_learners=max_num_weak_learners,
        hidden_dim=hidden_dim,
        tol=1e-5,
        patience=2,
        device=device
    )
    trainer.train(X_data, batch_size=128, n_inner_epochs=10, fit_new_learner_epochs=5, verbose=verbose)
    W_bin = trainer.get_binarized_adjacency(threshold=threshold)
    # Remove any residual cycles
    G_nx = nx.DiGraph(W_bin)
    remove_all_cycles(G_nx)
    final_adj = nx.to_numpy_array(G_nx, dtype=int)
    return final_adj, trainer

def flar(X_data, threshold=0.3, max_iter=5, max_num_weak_learners=50, hidden_dim=40,
         lambda_h=5.0, verbose=False):
    """
    Wrapper for running DAG‑Boosting.
    """
    return run_dagboost_method(X_data, threshold=threshold, max_iter=max_iter,
                               max_num_weak_learners=max_num_weak_learners,
                               hidden_dim=hidden_dim, lambda_h=lambda_h, verbose=verbose)

# **Comparison Against PC**

In [ ]:


# ------------------------------------------------
# 4. PC Method (using causal‑learn)
# ------------------------------------------------

def run_pc_method(X_data):
    """
    Run the PC algorithm from causal‑learn on data X_data.
    Based on the usage in the causal-learn docs, PC is invoked as:
         from causallearn.search.ConstraintBased.PC import pc
         result = pc(X_data, alpha=0.05)
    This function converts the resulting graph into a binary adjacency matrix,
    where an entry of 1 in est_adj[i,j] indicates an edge i -> j.
    """
    from causallearn.search.ConstraintBased.PC import pc
    try:
        # Run PC with a default significance level (alpha=0.05)
        pc_result = pc(X_data, alpha=0.05)
    except Exception as e:
        raise RuntimeError(f"PC algorithm failed with error: {e}")

    # Convert the result to an adjacency matrix.
    # Here we assume pc_result.G.graph is a NumPy array encoding the graph.
    pc_matrix = pc_result.G.graph
    d = pc_matrix.shape[0]
    est_adj = np.zeros((d, d), dtype=int)
    # For each pair, if pc_matrix[j,i]==1, then there is an edge i -> j.
    for i in range(d):
        for j in range(d):
            if pc_matrix[j, i] == 1:
                est_adj[i, j] = 1
    # Remove cycles if any
    G_nx = nx.DiGraph(est_adj)
    remove_all_cycles(G_nx)
    final_adj = nx.to_numpy_array(G_nx, dtype=int)
    return final_adj

# ------------------------------------------------
# 5. Main Comparison Loop: Testing Configurations
# ------------------------------------------------

if __name__ == '__main__':

    run_cell = True  # Set run_cell to True to execute
    if not run_cell:
        class StopExecutionWithMessage(Exception):
            def __init__(self, message):
                self.message = message
            def _render_traceback_(self):
                print(self.message)
                return []
        raise StopExecutionWithMessage("Execution halted.")

    # Experimental configurations
    var_settings = [100, 500]       # Number of variables
    dag_types = ['ER','SF']           # ER: Erdős–Rényi, SF: Scale-Free
    sem_types = ['linear','non-linear']
    noise_types = ['gaussian','exponential','laplace']  # Use "laplace" for Laplace noise
    n_samples = 500                    # Number of samples per experiment
    weight_scale = 1.0

    results = []

    for n_vars in var_settings:
        edge_prob = 2.0 / (n_vars - 1)  # Expected average degree ~2
        for dag_type in dag_types:
            # Generate the true DAG
            if dag_type == 'ER':
                true_adj = generate_erdos_renyi_dag(n_vars, edge_prob)
            elif dag_type == 'SF':
                true_adj = generate_scale_free_dag(n_vars)
            else:
                continue

            for sem_type in sem_types:
                for noise in noise_types:
                    config_str = f"n_vars={n_vars}, DAG={dag_type}, SEM={sem_type}, Noise={noise}"
                    print("\n[Config] " + config_str)
                    # Simulate data and obtain true weight matrix
                    if sem_type == 'linear':
                        X_data, W_true = simulate_sem_with_weights(true_adj, n_samples,
                                                                   noise_type=noise,
                                                                   weight_scale=weight_scale)
                    else:
                        X_data, W_true = simulate_non_linear_sem_with_weights(true_adj, n_samples,
                                                                              noise_type=noise,
                                                                              weight_scale=weight_scale,
                                                                              non_linear_fn=np.tanh)

                    # Create normalized data (standardization: zero mean, unit std)
                    X_data_norm = (X_data - X_data.mean(axis=0)) / (X_data.std(axis=0) + 1e-8)
                    # ---------------------------
                    # Run PC Method
                    try:
                        t0 = time.time()
                        est_adj_pc = run_pc_method(X_data)
                        runtime_pc = time.time() - t0
                        shd_pc = shd(true_adj, est_adj_pc)
                        sid_pc = sid(true_adj, est_adj_pc)
                        ate_rmse_pc = rmse_ate(W_true, est_adj_pc.astype(float))
                    except Exception as e:
                        runtime_pc = np.nan
                        shd_pc = np.nan
                        sid_pc = np.nan
                        ate_rmse_pc = np.nan
                        print("  PC method failed with error:", e)

                    results.append({
                        'Method': 'PC',
                        'n_vars': n_vars,
                        'DAG_type': dag_type,
                        'SEM_type': sem_type,
                        'Noise': noise,
                        'Runtime_sec': runtime_pc,
                        'SHD': shd_pc,
                        'SID': sid_pc,
                        'ATE_RMSE': ate_rmse_pc
                    })

                    # ---------------------------
                    # Run DAG‑Boosting (FLAR) Method
                    try:
                        t0 = time.time()
                        est_adj_dagboost, trainer = flar(X_data, threshold=0.3, verbose=False)
                        runtime_dagboost = time.time() - t0
                        shd_dagboost = shd(true_adj, est_adj_dagboost)
                        sid_dagboost = sid(true_adj, est_adj_dagboost)
                        ate_rmse_dagboost = rmse_ate(W_true, est_adj_dagboost.astype(float))
                    except Exception as e:
                        runtime_dagboost = np.nan
                        shd_dagboost = np.nan
                        sid_dagboost = np.nan
                        ate_rmse_dagboost = np.nan
                        print("  DAG‑Boosting method failed with error:", e)

                    results.append({
                        'Method': 'DAGBoost',
                        'n_vars': n_vars,
                        'DAG_type': dag_type,
                        'SEM_type': sem_type,
                        'Noise': noise,
                        'Runtime_sec': runtime_dagboost,
                        'SHD': shd_dagboost,
                        'SID': sid_dagboost,
                        'ATE_RMSE': ate_rmse_dagboost
                    })

    # Compile and print results as a DataFrame
    df_results = pd.DataFrame(results)
    print("\n===== Comparison Results =====")
    print(df_results)

    # Save the results DataFrame as CSV and JSON files
    output_csv_filename = "PC_results.csv"
    output_json_filename = "PC_results.json"

    # Save to CSV (with headers, no index)
    df_results.to_csv(output_csv_filename, index=False)
    print(f"Results saved to {output_csv_filename}")

    # Save to JSON in records format (one JSON object per line)
    df_results.to_json(output_json_filename, orient="records", lines=True)
    print(f"Results saved to {output_json_filename}")


[Config] n_vars=100, DAG=ER, SEM=linear, Noise=gaussian


  0%|          | 0/100 [00:00<?, ?it/s]

# **Comparison with fGES**

In [16]:
def run_ges_method(X_data):
    """
    Run the XGES algorithm on data X_data.

    Returns a binary adjacency matrix such that est_adj[i,j]=1 indicates edge i → j.
    """
    from xges import XGES
    import numpy as np
    import networkx as nx

    try:
        xges = XGES()
        pdag = xges.fit(X_data)
        dag = pdag.get_dag_extension()
        est_adj = dag.to_adjacency_matrix()
    except Exception as e:
        raise RuntimeError(f"XGES algorithm failed with error: {e}")

    # Remove any residual cycles.
    G_nx = nx.DiGraph(est_adj)
    remove_all_cycles(G_nx)
    final_adj = nx.to_numpy_array(G_nx, dtype=int)
    return final_adj


if __name__ == '__main__':

    run_cell = True  # Set run_cell to True to execute
    if not run_cell:
        class StopExecutionWithMessage(Exception):
            def __init__(self, message):
                self.message = message
            def _render_traceback_(self):
                print(self.message)
                return []
        raise StopExecutionWithMessage("Execution halted.")

    # Experimental configurations
    var_settings = [500]       # Number of variables
    dag_types = ['ER']#, 'SF']           # ER: Erdős–Rényi, SF: Scale‑Free
    sem_types = ['linear']#, 'non-linear']
    noise_types = ['gaussian']#, 'exponential', 'laplace']  # Use "laplace" for Laplace noise
    n_samples = 500                    # Number of samples per experiment
    weight_scale = 1.0

    results = []

    for n_vars in var_settings:
        # Set edge probability for an expected average degree of ~2
        edge_prob = 2.0 / (n_vars - 1)
        for dag_type in dag_types:
            # Generate the true DAG
            if dag_type == 'ER':
                true_adj = generate_erdos_renyi_dag(n_vars, edge_prob)
            elif dag_type == 'SF':
                true_adj = generate_scale_free_dag(n_vars)
            else:
                continue

            for sem_type in sem_types:
                for noise in noise_types:
                    config_str = f"n_vars={n_vars}, DAG={dag_type}, SEM={sem_type}, Noise={noise}"
                    print("\n[Config] " + config_str)
                    # Simulate data and obtain true weight matrix
                    if sem_type == 'linear':
                        X_data, W_true = simulate_sem_with_weights(true_adj, n_samples,
                                                                   noise_type=noise,
                                                                   weight_scale=weight_scale)
                    else:  # non-linear
                        X_data, W_true = simulate_non_linear_sem_with_weights(true_adj, n_samples,
                                                                              noise_type=noise,
                                                                              weight_scale=weight_scale,
                                                                              non_linear_fn=np.tanh)
                    # Create normalized data (standardization: zero mean, unit std)
                    X_data_norm = (X_data - X_data.mean(axis=0)) / (X_data.std(axis=0) + 1e-8)
                    # ---------------------------
                    # Run GES Method
                    try:
                        t0 = time.time()
                        est_adj_ges = run_ges_method(X_data)
                        runtime_ges = time.time() - t0
                        shd_ges = shd(true_adj, est_adj_ges)
                        sid_ges = sid(true_adj, est_adj_ges)
                        ate_rmse_ges = rmse_ate(W_true, est_adj_ges.astype(float))
                    except Exception as e:
                        runtime_ges = np.nan
                        shd_ges = np.nan
                        sid_ges = np.nan
                        ate_rmse_ges = np.nan
                        print("  GES method failed with error:", e)

                    results.append({
                        'Method': 'GES',
                        'n_vars': n_vars,
                        'DAG_type': dag_type,
                        'SEM_type': sem_type,
                        'Noise': noise,
                        'Runtime_sec': runtime_ges,
                        'SHD': shd_ges,
                        'SID': sid_ges,
                        'ATE_RMSE': ate_rmse_ges
                    })

                    # ---------------------------
                    # Run DAG‑Boosting (FLAR) Method
                    try:
                        t0 = time.time()
                        est_adj_dagboost, trainer = flar(X_data, threshold=0.3, verbose=False)
                        runtime_dagboost = time.time() - t0
                        shd_dagboost = shd(true_adj, est_adj_dagboost)
                        sid_dagboost = sid(true_adj, est_adj_dagboost)
                        ate_rmse_dagboost = rmse_ate(W_true, est_adj_dagboost.astype(float))
                    except Exception as e:
                        runtime_dagboost = np.nan
                        shd_dagboost = np.nan
                        sid_dagboost = np.nan
                        ate_rmse_dagboost = np.nan
                        print("  DAG‑Boosting method failed with error:", e)

                    results.append({
                        'Method': 'DAGBoost',
                        'n_vars': n_vars,
                        'DAG_type': dag_type,
                        'SEM_type': sem_type,
                        'Noise': noise,
                        'Runtime_sec': runtime_dagboost,
                        'SHD': shd_dagboost,
                        'SID': sid_dagboost,
                        'ATE_RMSE': ate_rmse_dagboost
                    })

    # Compile and print results as a DataFrame
    df_results = pd.DataFrame(results)
    print("\n===== Comparison Results =====")
    print(df_results)

    # Save the results DataFrame as CSV and JSON files
    output_csv_filename = "GES_500_results.csv"
    output_json_filename = "GES_500_results.json"

    # Save to CSV (with headers, no index)
    df_results.to_csv(output_csv_filename, index=False)
    print(f"Results saved to {output_csv_filename}")

    # Save to JSON in records format (one JSON object per line)
    df_results.to_json(output_json_filename, orient="records", lines=True)
    print(f"Results saved to {output_json_filename}")


[Config] n_vars=500, DAG=ER, SEM=linear, Noise=gaussian


XGES: Package `numba` not found. Falling back to the slower BICScorer implementation. Install `numba` for better performance with `pip install numba`.
XGES: Final score: -130135.16249911231



===== Comparison Results =====
     Method  n_vars DAG_type SEM_type     Noise  Runtime_sec  SHD   SID  \
0       GES     500       ER   linear  gaussian  1738.693927  287  3324   
1  DAGBoost     500       ER   linear  gaussian    25.805694  509  1000   

   ATE_RMSE  
0  0.139818  
1  0.053367  
Results saved to GES_500_results.csv
Results saved to GES_500_results.json


# **Comparison with SDCD**

In [3]:
!pip install sdcd
!pip install wandb==0.15.10
!pip install protobuf==3.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 29.0 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8792 sha256=727e404f847813c6f320e882ed304d2a5b71fe5df547b859f1fb8ac70d133df7
  Stored in directory: /root/.cache/pip/wheels/ea/b7/8b/84e94095ea418b9442f5abeba4ca7b0ad52d3fe7b69d6238a6
Successfully built pathtools
  Attempting uninstall: protobuf
    F

In [ ]:
from sdcd.models import SDCD
from sdcd.utils import create_intervention_dataset

def run_sdcd_method(n_vars):

  # Simulate Data
  from sdcd.simulated_data import random_model_gaussian_global_variance # For demonstration

  n = 500
  n_per_intervention = 50
  d = n_vars
  n_edges = int(d*(2.0 / (n_vars - 1)))

  true_causal_model = random_model_gaussian_global_variance(
      d,
      n_edges,
      dag_type="ER",
      scale=0.5,
      hard=True,
  )
  X_df = true_causal_model.generate_dataframe_from_all_distributions(
      n_samples_control=n,
      n_samples_per_intervention=n_per_intervention,
  )
  X_df.iloc[:, :-1] = (X_df.iloc[:, :-1] - X_df.iloc[:, :-1].mean()) / X_df.iloc[
      :, :-1
  ].std() # Normalize the data

  X_dataset = create_intervention_dataset(X_df, perturbation_colname="perturbation_label")
  model = SDCD()
  model.train(X_dataset)
  adj_matrix = model.get_adjacency_matrix(threshold=True)
  return adj_matrix

if __name__ == '__main__':

    run_cell = True  # Set this to True to execute the comparison
    if not run_cell:
        class StopExecution(Exception):
            def __init__(self, msg):
                self.msg = msg
            def _render_traceback_(self):
                print(self.msg)
                return []
        raise StopExecution("Execution halted.")

    # Experimental configurations
    var_settings = [100,500]       # Number of variables
    dag_types = ['ER', 'SF']           # DAG types: Erdős–Rényi, Scale‑Free
    sem_types = ['linear', 'non-linear']
    noise_types = ['gaussian', 'exponential', 'laplace']  # Use "laplace" for Laplace noise
    n_samples = 500                    # Samples per experiment
    weight_scale = 1.0

    results = []

    for n_vars in var_settings:
        edge_prob = 2.0 / (n_vars - 1)  # Expected average degree ~2
        for dag_type in dag_types:
            # Generate true DAG
            if dag_type == 'ER':
                true_adj = generate_erdos_renyi_dag(n_vars, edge_prob)
            elif dag_type == 'SF':
                true_adj = generate_scale_free_dag(n_vars)
            else:
                continue
            for sem_type in sem_types:
                for noise in noise_types:
                    config_str = f"n_vars={n_vars}, DAG={dag_type}, SEM={sem_type}, Noise={noise}"
                    print("\n[Config] " + config_str)
                    # Simulate data and get true weight matrix
                    if sem_type == 'linear':
                        X_data, W_true = simulate_sem_with_weights(true_adj, n_samples,
                                                                   noise_type=noise,
                                                                   weight_scale=weight_scale)
                    else:
                        X_data, W_true = simulate_non_linear_sem_with_weights(true_adj, n_samples,
                                                                              noise_type=noise,
                                                                              weight_scale=weight_scale,
                                                                              non_linear_fn=np.tanh)
                    # Create normalized data (standardization: zero mean, unit std)
                    X_data_norm = (X_data - X_data.mean(axis=0)) / (X_data.std(axis=0) + 1e-8)
                    # ---------------------------
                    # Run SDCD Method
                    try:
                        t0 = time.time()
                        est_adj_sdcd = run_sdcd_method(n_vars)
                        runtime_sdcd = time.time() - t0
                        shd_sdcd = shd(true_adj, est_adj_sdcd)
                        sid_sdcd = sid(true_adj, est_adj_sdcd)
                        ate_rmse_sdcd = rmse_ate(W_true, est_adj_sdcd.astype(float))
                    except Exception as e:
                        runtime_sdcd = np.nan
                        shd_sdcd = np.nan
                        sid_sdcd = np.nan
                        ate_rmse_sdcd = np.nan
                        print("  SDCD method failed with error:", e)

                    results.append({
                        'Method': 'SDCD',
                        'n_vars': n_vars,
                        'DAG_type': dag_type,
                        'SEM_type': sem_type,
                        'Noise': noise,
                        'Runtime_sec': runtime_sdcd,
                        'SHD': shd_sdcd,
                        'SID': sid_sdcd,
                        'ATE_RMSE': ate_rmse_sdcd
                    })

                    # ---------------------------
                    # Run DAG‑Boosting (FLAR) Method
                    try:
                        t0 = time.time()
                        est_adj_dagboost, trainer = (X_data, threshold=0.3, verbose=False)
                        runtime_dagboost = time.time() - t0
                        shd_dagboost = shd(true_adj, est_adj_dagboost)
                        sid_dagboost = sid(true_adj, est_adj_dagboost)
                        ate_rmse_dagboost = rmse_ate(W_true, est_adj_dagboost.astype(float))
                    except Exception as e:
                        runtime_dagboost = np.nan
                        shd_dagboost = np.nan
                        sid_dagboost = np.nan
                        ate_rmse_dagboost = np.nan
                        print("  DAG‑Boosting method failed with error:", e)

                    results.append({
                        'Method': 'DAGBoost',
                        'n_vars': n_vars,
                        'DAG_type': dag_type,
                        'SEM_type': sem_type,
                        'Noise': noise,
                        'Runtime_sec': runtime_dagboost,
                        'SHD': shd_dagboost,
                        'SID': sid_dagboost,
                        'ATE_RMSE': ate_rmse_dagboost
                    })

    df_results = pd.DataFrame(results)
    print("\n===== Comparison Results =====")
    print(df_results)

    # Save the results DataFrame as CSV and JSON files
    output_csv_filename = "SDCD_results.csv"
    output_json_filename = "SDCD_results.json"

    # Save to CSV (with headers, no index)
    df_results.to_csv(output_csv_filename, index=False)
    print(f"Results saved to {output_csv_filename}")

    # Save to JSON in records format (one JSON object per line)
    df_results.to_json(output_json_filename, orient="records", lines=True)
    print(f"Results saved to {output_json_filename}")

# **Comparison with SAM**

In [10]:
# ------------------------------------------------
# 4. SAM Method (using the cdt Package)
# ------------------------------------------------

#import multiprocessing as mp
#mp.set_start_method('spawn', force=True) # Before your SAM code
import cdt

def run_sam_method(X_data, nruns=1):
    """
    Run the SAM algorithm from cdt.
    Convert the simulated data (NumPy array) into a pandas DataFrame,
    then run SAM. The output graph is converted to a binary adjacency matrix
    where an edge i → j is denoted by a 1 at position [i, j].
    """
    try:
        from cdt.causality.graph import SAM
        import cdt
    except ImportError:
        raise ImportError("cdt package is required for SAM. Install with 'pip install cdt'.")

    cdt.SETTINGS.GPU = 0 # use CPU
    data_df = pd.DataFrame(X_data)
    sam = SAM(nruns=nruns) #sam = SAM(nruns=1)
    # SAM's output is a graph; we assume output_graph.edges() returns directed edges.
    output_graph = sam.predict(data_df)
    d = X_data.shape[1]
    est_adj = np.zeros((d, d), dtype=int)
    # For each edge in the graph, assume it is directed: if there is an edge from u to v,
    # then set est_adj[u, v] = 1.
    for (u, v) in output_graph.edges():
        est_adj[u, v] = 1
    # Optionally remove cycles.
    G_nx = nx.DiGraph(est_adj)
    remove_all_cycles(G_nx)
    final_adj = nx.to_numpy_array(G_nx, dtype=int)
    return final_adj

# ------------------------------------------------
# 5. Main Comparison Loop: Testing Configurations
# ------------------------------------------------

if __name__ == '__main__':

    run_cell = True  # Set this to True to execute the comparison
    if not run_cell:
        class StopExecution(Exception):
            def __init__(self, msg):
                self.msg = msg
            def _render_traceback_(self):
                print(self.msg)
                return []
        raise StopExecution("Execution halted.")

    # Experimental configurations
    var_settings = [100,500]      # Number of variables
    dag_types = ['ER', 'SF']           # DAG types: Erdős–Rényi, Scale‑Free
    sem_types = ['linear', 'non-linear']
    noise_types = ['gaussian', 'exponential', 'laplace']  # Use "laplace" for Laplace noise
    n_samples = 500                    # Samples per experiment
    weight_scale = 1.0

    results = []

    for n_vars in var_settings:
        edge_prob = 2.0 / (n_vars - 1)  # Expected average degree ~2
        for dag_type in dag_types:
            # Generate true DAG
            if dag_type == 'ER':
                true_adj = generate_erdos_renyi_dag(n_vars, edge_prob)
            elif dag_type == 'SF':
                true_adj = generate_scale_free_dag(n_vars)
            else:
                continue
            for sem_type in sem_types:
                for noise in noise_types:
                    config_str = f"n_vars={n_vars}, DAG={dag_type}, SEM={sem_type}, Noise={noise}"
                    print("\n[Config] " + config_str)
                    # Simulate data and get true weight matrix
                    if sem_type == 'linear':
                        X_data, W_true = simulate_sem_with_weights(true_adj, n_samples,
                                                                   noise_type=noise,
                                                                   weight_scale=weight_scale)
                    else:
                        X_data, W_true = simulate_non_linear_sem_with_weights(true_adj, n_samples,
                                                                              noise_type=noise,
                                                                              weight_scale=weight_scale,
                                                                              non_linear_fn=np.tanh)
                    # Create normalized data (standardization: zero mean, unit std)
                    X_data_norm = (X_data - X_data.mean(axis=0)) / (X_data.std(axis=0) + 1e-8)
                    # ---------------------------
                    # Run SAM Method
                    try:
                        t0 = time.time()
                        est_adj_sam = run_sam_method(X_data, nruns=1)
                        runtime_sam = time.time() - t0
                        shd_sam = shd(true_adj, est_adj_sam)
                        sid_sam = sid(true_adj, est_adj_sam)
                        ate_rmse_sam = rmse_ate(W_true, est_adj_sam.astype(float))
                    except Exception as e:
                        runtime_sam = np.nan
                        shd_sam = np.nan
                        sid_sam = np.nan
                        ate_rmse_sam = np.nan
                        print("  SAM method failed with error:", e)

                    results.append({
                        'Method': 'SAM',
                        'n_vars': n_vars,
                        'DAG_type': dag_type,
                        'SEM_type': sem_type,
                        'Noise': noise,
                        'Runtime_sec': runtime_sam,
                        'SHD': shd_sam,
                        'SID': sid_sam,
                        'ATE_RMSE': ate_rmse_sam
                    })

                    # ---------------------------
                    # Run DAG‑Boosting (FLAR) Method
                    try:
                        t0 = time.time()
                        est_adj_dagboost, trainer = flar(X_data, threshold=0.3, verbose=False)
                        runtime_dagboost = time.time() - t0
                        shd_dagboost = shd(true_adj, est_adj_dagboost)
                        sid_dagboost = sid(true_adj, est_adj_dagboost)
                        ate_rmse_dagboost = rmse_ate(W_true, est_adj_dagboost.astype(float))
                    except Exception as e:
                        runtime_dagboost = np.nan
                        shd_dagboost = np.nan
                        sid_dagboost = np.nan
                        ate_rmse_dagboost = np.nan
                        print("  DAG‑Boosting method failed with error:", e)

                    results.append({
                        'Method': 'DAGBoost',
                        'n_vars': n_vars,
                        'DAG_type': dag_type,
                        'SEM_type': sem_type,
                        'Noise': noise,
                        'Runtime_sec': runtime_dagboost,
                        'SHD': shd_dagboost,
                        'SID': sid_dagboost,
                        'ATE_RMSE': ate_rmse_dagboost
                    })

    df_results = pd.DataFrame(results)
    print("\n===== Comparison Results =====")
    print(df_results)

    # Save the results DataFrame as CSV and JSON files
    output_csv_filename = "SAM_results.csv"
    output_json_filename = "SAM_results.json"

    # Save to CSV (with headers, no index)
    df_results.to_csv(output_csv_filename, index=False)
    print(f"Results saved to {output_csv_filename}")

    # Save to JSON in records format (one JSON object per line)
    df_results.to_json(output_json_filename, orient="records", lines=True)
    print(f"Results saved to {output_json_filename}")


[Config] n_vars=50, DAG=ER, SEM=linear, Noise=gaussian


  1%|▏         | 58/4000 [00:15<17:36,  3.73it/s, disc=0.00417, gen=-1.1, regul_loss=47.5, tot=-7.26]


KeyboardInterrupt: 